In [1]:
import objaverse
import multiprocessing
import shutil
import chromadb
from flask import Flask, request, jsonify
from flask_cors import CORS
import urllib.parse

uids = objaverse.load_uids()
chroma_client = chromadb.PersistentClient(path="./thirty-thousand.db")
print(chroma_client.list_collections())
collection = chroma_client.get_collection(name="my_collection")

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route("/", methods=["POST"])
def handle_post():
    text_data = request.data.decode("utf-8")
    print("user: " + text_data)
    response = handle_prompt(text_data)
    return response

def handle_prompt(user_query):
    results = collection.query(query_texts=[user_query], n_results=1)
    # print(results)
    top_id = results["ids"][0]
    
    # print(objaverse.load_annotations(uids=top_id))
    annotation = objaverse.load_annotations(uids=top_id)
    thumbnail = annotation[0]
    print(thumbnail)
    
    
    # object_path = download_objects(top_id)
    # formatted_path = format_path_to_uri(object_path)
    # return formatted_path

def download_objects(ids):
    print("ids: " + str(ids))
    objects = objaverse.load_objects(uids=ids)
    object_path = list(objects.values())[0]
    return object_path

def format_path_to_uri(path):
    # Convert backslashes to forward slashes and prepend the file URI scheme
    uri_path = 'file:///' + path.replace("\\", "/")
    return uri_path

# For testing purposes
# print(handle_prompt("green chair"))


# app.run(debug=True, port=5001)

# curl -X POST -d "chair" http://localhost:5001/





In [60]:
import json

results = collection.query(query_texts=["pikachu"], n_results=3)

ids = results["ids"][0]
annotations = objaverse.load_annotations(uids=ids)

generated_descriptions = results["documents"][0]
titles = []
descriptions = []
thumbnails_url = []

for annotation in iter(annotations.values()):
    print(annotation)
    title = annotation["name"]
    titles.append(title)
    description = annotation["description"]
    descriptions.append(description)
    thumbails = annotation["thumbnails"]["images"]    
    first_thumbnail = thumbails[0]
    first_thumbnail_url = first_thumbnail["url"]
    thumbnails_url.append(first_thumbnail_url)

print(generated_descriptions)
print(titles)
print(descriptions)
print(thumbnails_url)


combined_data = [
    {
        "generated_description": gen_desc,
        "title": title,
        "description": desc,
        "thumbnail_url": thumb_url
    }
    for gen_desc, title, desc, thumb_url in zip(generated_descriptions, titles, descriptions, thumbnails_url)
]

# Convert the list of dictionaries to a JSON string
json_data = json.dumps(combined_data, indent=4)

# Print the JSON data
print(json_data)
# generate json

